In [1]:
import io
import zipfile
import frontmatter
import requests

In [4]:
doc_extensions = {'md', 'mdx'}
code_extensions = {'py', 'sql', 'java', 'ipynb'}

extensions = doc_extensions | code_extensions

def read_repo_data(repo_owner, repo_name):
    """
    Download and parse all markdown files from a GitHub repository.
    
    Args:
        repo_owner: GitHub username or organization
        repo_name: Repository name
    
    Returns:
        List of dictionaries containing file content and metadata
    """
    prefix = 'https://codeload.github.com' 
    url = f'{prefix}/{repo_owner}/{repo_name}/zip/refs/heads/main'
    resp = requests.get(url)
    
    if resp.status_code != 200:
        raise Exception(f"Failed to download repository: {resp.status_code}")

    repository_data = []
    zf = zipfile.ZipFile(io.BytesIO(resp.content))
    
    for file_info in zf.infolist():
        filepath = file_info.filename
        filepath_lower = filepath.lower()

        if filepath_lower.endswith('/'):
            continue

        filename = filepath_lower.split('/')[-1]

        if filename.startswith('.'):
            continue

        ext = filename.split('.')[-1]

        if ext not in extensions:
            continue

        filepath_edited = filepath.split('/', maxsplit=1)[1]

        try:
            with zf.open(file_info) as f_in:
                content = f_in.read().decode('utf-8', errors='ignore')
                if ext in doc_extensions:
                    post = frontmatter.loads(content)
                    data = post.to_dict()
                    data['filename'] = filepath_edited
                elif ext in code_extensions:
                    data = {
                        'code': True,
                        'content': content,
                        'filename': filepath_edited
                    }

                repository_data.append(data)
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            continue

    zf.close()
    return repository_data

In [5]:
my_repo = read_repo_data('Adh101','Geospatial-Data-Science-on-NYC-Bike-Trip-Data')
print(my_repo)
print(f'My Repo:{len(my_repo)}')

[{'content': '# Geospatial Data Science on NYC Bike Trip Data (PostgreSQL + PostGIS + GeoPandas)\n\nA **PostgreSQL/PostGIS–first** project that ingests NYC bike-trip data, answers core business questions, bins usage by time, prepares spatial data (SRIDs, point geometry creation), performs tract-level spatial joins/aggregations, and runs proximity/buffer analysis for operations. Further, **GeoPandas** and **Folium** are used to visualize the aggregated data in Python.\n\n## 📚 Reference\n\n- Ma, Maggie. *Hands-On PostgreSQL Project: Spatial Data Science*. LinkedIn Learning.  \n  [Course link](https://www.linkedin.com/learning/hands-on-postgresql-project-spatial-data-science)\n\n---\n\n## 📂 Repository Contents\n\n```\n├── analysis_exports/\n│   ├── half_hour_interval_start_counts.csv\n│   ├── spatio_temporal_visualization_05_01_202509171746.csv\n│   ├── top_stations_buffer.csv\n│   └── trip_count_by_census_tract.csv\n├── data/\n│   ├── nyct2020.geojson\n│   ├── stations.csv\n│   └── trip_